
# Storytime


Before running or if using a different scenario, the user needs to  
make the below changes.  
* Open the scenario 
* Update the file path
* Set the animation time & time step
* In createfolders() decide how the files should be structured  

The goal in this script is to simplify generating eoir synthetic data  
This is done in a few steps 

## _main()_

main() kicks everything off. Using the scenario time, it calculates the  
number of total frames between start and stop. This will be the number  
of times eoirdata() steps through the scenario. The loop jumps to the  
beginning of the scenario then cycles through the following cloud and  
atmospheric parameters.   

|Clouds| . |Atmosphere|
|:-----|:-:|---------:|
|Off   | . |Off       |
|On    | . |Off       |
|On    | . |Simple       |
|On    | . |MODTRAN   |

Once the parameters are set, it dives into eoirdata()

## _eoirdata()_  

eoirdata() uses connect commands to generate the bitmap and the  
raw data. In the same line, the data is stored into it's respective  
folder (see createfolders()). This function then steps forward in time  
and does the whole thing again. The range, or number of iterations is   
set by the number of frames. Once it reaches the end time the function  
jumps out. 

## _createfolders()_  
Create the folders that eoirdata() will save to. It will first check  
if the directories exist and decide if they need to be generated.  

## _Misc_

Replace the manually entered information and let that be entered in as as function

if using custom file, then use  
root.ExecuteCommand('EOIR */ CloudData SetValue MyConstDataSet Altitude 12    
PercentCloudCoverConst 1 PercentCloudCoverFile   
"C:\ProgramData\AGI\STK 12\STKData\VO\Clouds\clouds_2020126.100000_2020127.160000.cld  
EmissivityConst .5 TemperatureConst 233 ForwardScatterConst .1)  


* attach to scenario PID  


# Connect Command Reference

Atmosphere Key
* 0 - off.
* 1 - Simple atmosphere.
* 2 - MODTRAN atmosphere.

Atmosphere Simple Parameters
* Aerosol Models: Rural
* Visibility: 25.00 km
* Humidity: 45.8 %
    
Cloud Key
* On - turns on clouds.
* Off - turns off clouds.

Cloud Parameters
* Altitude: 12 km
* Forward Scatter: 10%
* Back Scatter: 80%
* Constant Coverage: 75%
* Constant Emissivity and Temperature
* Emissivity: 50%
* Temperature: 233K


In [192]:
import os 
import csv
import time
import itertools
from datetime import timedelta
from datetime import datetime
from agi.stk12.stkdesktop import STKDesktop
from agi.stk12.stkobjects import *

# Open EOIR_Clouds_AirToAir.sc
# Connect to the scenario
# Get the IAgStkObjectRoot interface 

stk      = STKDesktop.AttachToApplication()  
root     = stk.Root                           
scenario = root.CurrentScenario 

# Set the scenario animation properties
animation               = scenario.Animation
animation.StartTime     = '5 May 2020 11:41:30.000'
animation.EnableAnimCycleTime = True
animation.AnimCycleTime = '5 May 2020 11:46:30.000'
animation.AnimCycleType = AgEScEndLoopType.eLoopAtTime
animation.AnimStepValue = 0.5


In [193]:
def createfolders(filepath,data,sensor,ftype,cld,atm):

    newdir = os.mkdir("{}output/".format(filepath))

    # Directory structure based on generated data
    for (a,b,c) in itertools.product(data,ftype,sensor): 
        for (x,y) in itertools.zip_longest(cld,atm):
            name = "{}_{}_{}_Cld{}_Atm{}".format(a, b, c, x, y)
            # print("{}output/{}".format(filepath,name))
            os.mkdir("{}output/{}".format(filepath,name))


In [242]:
def findObjects():
    # Ideally this should be able to look through ALL objects in the 
    # scenario and find ALL child objects... but anyways here we are. 
    parentObj = scenario.Children.GetElements(AgESTKObjectType.eAircraft)

    parClass = []
    parName  = []
    chlClass = []
    chlName  = []

    # the scenario has multiple aircraft, sweep through them all
    for i in range(parentObj.Count): 
        if parentObj.Item(i).HasChildren == True:
            # capture the identity of the parent object that has children
            parClass.append(parentObj.Item(i).ClassName)
            parName.append(parentObj.Item(i).InstanceName)
            for j in range(parentObj.Item(i).Children.Count):
                # find number of child objects
                # capture relevant properties 
                chlClass.append(parentObj.Item(i).Children
                    .GetElements(AgESTKObjectType.eSensor).Item(j).ClassName)
                chlName.append(parentObj.Item(i).Children
                    .GetElements(AgESTKObjectType.eSensor).Item(j).InstanceName)
        else:
            continue

    return parClass,parName,chlClass,chlName 

In [194]:
def eoirdata(filepath,x,y,z):

    # EOIR connect commands SaveSceneImage & SaveSceneRawData 
    command_bmp_LWIR    = ("EOIRDetails */Aircraft/SouthBound/Sensor/LWIR SaveSceneImage "\
        "\"{}output/raw_bitmap_LWIR".format(filepath))
    command_txt_LWIR    = ("EOIRDetails */Aircraft/SouthBound/Sensor/LWIR SaveSceneRawData "\
        "\"{}output/raw_txt_LWIR".format(filepath))
    command_bmp_Visible = ("EOIRDetails */Aircraft/SouthBound/Sensor/Visible SaveSceneImage "\
        "\"{}output/raw_bitmap_Visible".format(filepath))
    command_txt_Visible = ("EOIRDetails */Aircraft/SouthBound/Sensor/Visible SaveSceneRawData "\
        "\"{}output/raw_txt_Visible".format(filepath))
    
    # Loop through the entire scenario 
    for i in range(z):
        print("{}_Cld{}_Atm{}/{}.bmp\"".format(command_bmp_LWIR,x,y,i))
        print("{}_Cld{}_Atm{}/{}.txt\"".format(command_txt_LWIR,x,y,i))
        print("{}_Cld{}_Atm{}/{}.bmp\"".format(command_bmp_Visible,x,y,i))    
        print("{}_Cld{}_Atm{}/{}.txt\"".format(command_txt_Visible,x,y,i))
        # root.ExecuteCommand("{}_Cld{}_Atm{}/{}.bmp\"".format(command_bmp_LWIR,x,y,i))
        # root.ExecuteCommand("{}_Cld{}_Atm{}/{}.txt\"".format(command_txt_LWIR,x,y,i))
        # root.ExecuteCommand("{}_Cld{}_Atm{}/{}.bmp\"".format(command_bmp_Visible,x,y,i))    
        # root.ExecuteCommand("{}_Cld{}_Atm{}/{}.txt\"".format(command_txt_Visible,x,y,i))
        root.StepForward()


In [225]:
def main():

    # Measure elapsed time for data generation
    start = time.time() 
    
    # Total frames generated based on scenario animation time/timestep
    d1 = datetime.strptime(animation.StartTime,     "%d %b %Y %H:%M:%S.%f")
    d2 = datetime.strptime(animation.AnimCycleTime, "%d %b %Y %H:%M:%S.%f")
    #k  = int(timedelta.total_seconds(d2 - d1)/animation.AnimStepValue)
    k  = 4 # run for testing

    parClass,parName,chlClass,sensor  = findObjects()

    ########
    # USER DEFINED 
    # File path for the output data 
    filepath = 'C:/Users/nkazmi/Documents/STK 12/EOIR_Clouds_AirToAir/'

    data    = ['raw','processed'] # the state of the data
    ftype   = ['bitmap','txt']    # exported data format

    cld     = ['Off','On','On']
    atm     = [0,0,1]
    ########

    # Check if the save to directories exist 
    # Create them before generating data
    isExist = os.path.exists("{}output/".format(filepath))

    if isExist == False:
        print('Creating Directories...')
        createfolders(filepath,data,sensor,ftype,cld,atm)
    else:
        print('Directories Exist')

    # Jump to the beginning of the animation time
    # Set EOIR Cloud and Atmosphere properties
    # Animate and generate data for each timestep 
    for (i,j) in itertools.zip_longest(cld,atm):
        root.Rewind()
        root.ExecuteCommand("EOIR */ SetAtmosphere Clouds {}".format(i))
        root.ExecuteCommand("EOIR */ SetAtmosphere Mode {}".format(j))
        eoirdata(filepath,i,j,k)
    

    end = time.time()
    print("Elapsed Time (hr): " + str((end - start)/(3600)))

if __name__ == '__main__':
    main()

Directories Exist
EOIRDetails */Aircraft/SouthBound/Sensor/LWIR SaveSceneImage "C:/Users/nkazmi/Documents/STK 12/EOIR_Clouds_AirToAir/output/raw_bitmap_LWIR_CldOff_Atm0/0.bmp"
EOIRDetails */Aircraft/SouthBound/Sensor/LWIR SaveSceneRawData "C:/Users/nkazmi/Documents/STK 12/EOIR_Clouds_AirToAir/output/raw_txt_LWIR_CldOff_Atm0/0.txt"
EOIRDetails */Aircraft/SouthBound/Sensor/Visible SaveSceneImage "C:/Users/nkazmi/Documents/STK 12/EOIR_Clouds_AirToAir/output/raw_bitmap_Visible_CldOff_Atm0/0.bmp"
EOIRDetails */Aircraft/SouthBound/Sensor/Visible SaveSceneRawData "C:/Users/nkazmi/Documents/STK 12/EOIR_Clouds_AirToAir/output/raw_txt_Visible_CldOff_Atm0/0.txt"
EOIRDetails */Aircraft/SouthBound/Sensor/LWIR SaveSceneImage "C:/Users/nkazmi/Documents/STK 12/EOIR_Clouds_AirToAir/output/raw_bitmap_LWIR_CldOff_Atm0/1.bmp"
EOIRDetails */Aircraft/SouthBound/Sensor/LWIR SaveSceneRawData "C:/Users/nkazmi/Documents/STK 12/EOIR_Clouds_AirToAir/output/raw_txt_LWIR_CldOff_Atm0/1.txt"
EOIRDetails */Aircraft/So

In [257]:
def eoirdatatest(parClass,parName,chlClass,sensor,filepath,x,y):

    for i in range(len(parClass)):
        for j in range(len(chlClass)):
            print("EOIRDetails */{}/{}/{}/{} SaveSceneImage \"{}output/raw_bitmap_{}_Cld{}_Atm{}/\""
                .format(parClass[i],parName[i],chlClass[j],sensor[j],filepath,sensor[j],x,y))
            print("EOIRDetails */{}/{}/{}/{} SaveSceneRawData \"{}output/raw_txt_{}_Cld{}_Atm{}/\""
                .format(parClass[i],parName[i],chlClass[j],sensor[j],filepath,sensor[j],x,y))


filepath = 'C:/Users/nkazmi/Documents/STK 12/EOIR_Clouds_AirToAir/'
cld      = ['Off','On','On']
atm      = [0,0,1]

parClass,parName,chlClass,sensor  = findObjects()
for (i,j) in itertools.zip_longest(cld,atm):
    eoirdatatest(parClass,parName,chlClass,sensor,filepath,i,j)

EOIRDetails */Aircraft/SouthBound/Sensor/LWIR SaveSceneImage "C:/Users/nkazmi/Documents/STK 12/EOIR_Clouds_AirToAir/output/raw_bitmap_LWIR_CldOff_Atm0/"
EOIRDetails */Aircraft/SouthBound/Sensor/LWIR SaveSceneRawData "C:/Users/nkazmi/Documents/STK 12/EOIR_Clouds_AirToAir/output/raw_txt_LWIR_CldOff_Atm0/"
EOIRDetails */Aircraft/SouthBound/Sensor/Visible SaveSceneImage "C:/Users/nkazmi/Documents/STK 12/EOIR_Clouds_AirToAir/output/raw_bitmap_Visible_CldOff_Atm0/"
EOIRDetails */Aircraft/SouthBound/Sensor/Visible SaveSceneRawData "C:/Users/nkazmi/Documents/STK 12/EOIR_Clouds_AirToAir/output/raw_txt_Visible_CldOff_Atm0/"
EOIRDetails */Aircraft/SouthBound/Sensor/LWIR SaveSceneImage "C:/Users/nkazmi/Documents/STK 12/EOIR_Clouds_AirToAir/output/raw_bitmap_LWIR_CldOn_Atm0/"
EOIRDetails */Aircraft/SouthBound/Sensor/LWIR SaveSceneRawData "C:/Users/nkazmi/Documents/STK 12/EOIR_Clouds_AirToAir/output/raw_txt_LWIR_CldOn_Atm0/"
EOIRDetails */Aircraft/SouthBound/Sensor/Visible SaveSceneImage "C:/Users/n